In [202]:
import pandas as pd
import pickle
import statsmodels.api as sm
import numpy as np
from getData import *

In [545]:
#USE POSTSEASON DADTA
with open(f'data/endofseasonratings/2023offense.pkl','rb') as f:
            offense = pickle.load(f)
with open(f'data/endofseasonratings/2023defense.pkl','rb') as f:
            defense = pickle.load(f)

#USE PRESEASON DATA
#with open(f'data/preseasonratings/2023offense.pkl','rb') as f:
#            offense = pickle.load(f)[2023]
#with open(f'data/preseasonratings/2023defense.pkl','rb') as f:
#            defense = pickle.load(f)[2023]

In [546]:
use_columns = [
    'pregame_home_off_ypp',
    'pregame_away_def_ypp', 'pregame_away_off_ypp', 'pregame_home_def_ypp',
     'pregame_home_off_TD',
    'pregame_away_def_TD', 'pregame_away_off_TD', 'pregame_home_def_TD',
    'pregame_home_off_TO', 'pregame_away_def_TO', 'pregame_away_off_TO',
    'pregame_home_def_TO', 'pregame_home_off_MC', 'pregame_away_def_MC',
    'pregame_away_off_MC', 'pregame_home_def_MC', 'pregame_home_off_FC',
    'pregame_away_def_FC', 'pregame_away_off_FC', 'pregame_home_def_FC',
    'pregame_home_off_HV',
    'pregame_away_def_HV', 'pregame_away_off_HV', 'pregame_home_def_HV',
    'pregame_home_off_PA', 'pregame_away_def_PA',
       'pregame_away_off_PA', 'pregame_home_def_PA', 'pregame_home_off_RU',
       'pregame_away_def_RU', 'pregame_away_off_RU', 'pregame_home_def_RU',
    'pregame_home_off_FGA','pregame_away_def_FGA', 'pregame_away_off_FGA', 'pregame_home_def_FGA',
    'pregame_home_off_FG','pregame_away_def_FG', 'pregame_away_off_FG', 'pregame_home_def_FG',
    'pregame_home_off_FD','pregame_away_def_FD', 'pregame_away_off_FD', 'pregame_home_def_FD',
     'HFA'
]

more_columns = ['pregame_home_off_ypp-pregame_away_def_ypp',
 'pregame_home_off_TD-pregame_away_def_TD',
 'pregame_home_off_TO-pregame_away_def_TO',
 'pregame_home_off_MC-pregame_away_def_MC',
 'pregame_home_off_FC-pregame_away_def_FC',
 'pregame_home_off_HV-pregame_away_def_HV',
 'pregame_home_off_PA-pregame_away_def_PA',
 'pregame_home_off_RU-pregame_away_def_RU',
 'pregame_home_off_FGA-pregame_away_def_FGA',
 'pregame_home_off_FG-pregame_away_def_FG',
 'pregame_home_off_FD-pregame_away_def_FD',
 'pregame_home_def_ypp-pregame_away_off_ypp',
 'pregame_home_def_TD-pregame_away_off_TD',
 'pregame_home_def_TO-pregame_away_off_TO',
 'pregame_home_def_MC-pregame_away_off_MC',
 'pregame_home_def_FC-pregame_away_off_FC',
 'pregame_home_def_HV-pregame_away_off_HV',
 'pregame_home_def_PA-pregame_away_off_PA',
 'pregame_home_def_RU-pregame_away_off_RU',
 'pregame_home_def_FGA-pregame_away_off_FGA',
 'pregame_home_def_FG-pregame_away_off_FG',
 'pregame_home_def_FD-pregame_away_off_FD']

In [547]:
offenseratings = pd.DataFrame().from_dict({team:[offense[i][team] for i in offense.keys()]  for i in offense.keys() for team in offense[i].keys()},
                        orient='index',columns=[x for x in use_columns if x.find("home_off")>0])
defenseratings = pd.DataFrame().from_dict({team:[defense[i][team] for i in defense.keys()]  for i in defense.keys() for team in defense[i].keys()},
                        orient='index',columns=[x for x in use_columns if x.find("home_def")>0])
ratings = pd.merge(offenseratings,defenseratings,how='left',left_index=True,right_index=True)
ratings.loc['fcs']=1200
ratingsaway=ratings.copy()
ratingsaway.columns = [x.replace("_home","_away") for x in ratings.columns]

In [630]:
ratings[ratings.index.isin(['Oregon','Washington'])][['pregame_home_off_ypp','pregame_home_off_HV','pregame_home_off_PA','pregame_home_off_RU','pregame_home_def_HV','pregame_home_def_PA','pregame_home_def_RU']]


,pregame_home_off_ypp,pregame_home_off_HV,pregame_home_off_PA,pregame_home_off_RU,pregame_home_def_HV,pregame_home_def_PA,pregame_home_def_RU
Washington,1697,1725,1680,1669,1549,1588,1571
Oregon,1790,1822,1767,1775,1628,1631,1623


In [548]:
pointmodel = pd.read_pickle("models/margin_predict.pickle")

In [549]:
# Make A Prediction
def predgame(homet,awayt,hfa=0):
    #add "fcs"
    if homet not in ratings.index:
        homet='fcs'
    if awayt not in ratings.index:
        awayt='fcs'
    
    makeprediction = pd.concat([ratings[ratings.index==homet].reset_index(),ratingsaway[ratingsaway.index==awayt].reset_index()],axis=1)
    #Create Interaction Columns
    homeoff = [c for c in makeprediction.columns if c.find("home_off")>0]
    awayoff = [c for c in makeprediction.columns if c.find("away_off")>0]
    homedef = [c for c in makeprediction.columns if c.find("home_def")>0]
    awaydef = [c for c in makeprediction.columns if c.find("away_def")>0]
    
    #morecols = []
    
    for a,b in zip(homeoff,awaydef):
        newstr = a+'-'+b
        #morecols.append(newstr)
        makeprediction[newstr]=makeprediction[a]-makeprediction[b]+1500
    for a,b in zip(homedef,awayoff):
        newstr = a+'-'+b
        makeprediction[newstr]=makeprediction[a]-makeprediction[b]+1500
        #morecols.append(newstr)
    
    makeprediction['HFA']=hfa
    makeprediction['const']=1
    return pointmodel.predict(makeprediction[['const']+use_columns+morecols]).iloc[0]

In [550]:
pointmodel.predict(makeprediction[['const']+use_columns+morecols]).iloc[0]
predgame('Michigan','Oregon')

-1.084085669332751

In [551]:
predgame('LSU','Grambling')

53.00663181307304

In [552]:
predgame('Michigan','Washington')

8.266386483066555

# Get Games and Prepare Data

In [553]:
games = getData('games',{'year':2023})

In [554]:
gamestest = games.query("(home_division=='fbs')or(away_division=='fbs')").query("week<14")#[['week','neutral_site','home_team','away_team','home_points','away_points']]
gamestest['HFA']=(gamestest['neutral_site']==False).astype(int)
gamestest['prediction']=gamestest.apply(lambda row: predgame(row['home_team'],row['away_team'],row['HFA']),axis=1)

In [555]:
gamestest = gamestest[['week','neutral_site','home_team','away_team','home_points','away_points','prediction']]\
.assign(margin = lambda x: x['home_points']-x['away_points'])

In [556]:
recap = pd.concat([(
    gamestest[['home_team','prediction','margin']].rename(columns={'home_team':'team'})
    .assign(predictedwin = lambda x: (x['prediction']>0).astype(int))
    .assign(win = lambda x: (x['margin']>0).astype(int))
    .assign(predictedloss = lambda x: (x['prediction']<0).astype(int))
    .assign(loss = lambda x: (x['margin']<0).astype(int))
),
(
    gamestest[['away_team','prediction','margin']].rename(columns={'away_team':'team'})
    .assign(predictedwin = lambda x: (x['prediction']<0).astype(int))
    .assign(win = lambda x: (x['margin']<0).astype(int))
    .assign(predictedloss = lambda x: (x['prediction']>0).astype(int))
    .assign(loss = lambda x: (x['margin']>0).astype(int))
)]).groupby('team')[['predictedwin','win','predictedloss','loss']].sum().reset_index()

# Simulate Seasons

In [658]:
#EXCLUDES CONFERENCE CHAMPIONSHIP GAMES
gamesim = gamestest.copy().query("week<14")

listofsim = pd.DataFrame()

In [659]:

for num in range(5000):
    gamesim['adjustment']=np.random.normal(0,14,len(gamesim))
    gamesim['newprediction']=gamesim['prediction']+gamesim['adjustment']
    
    simulation = pd.concat([(
        gamesim[['home_team','newprediction']].rename(columns={'home_team':'team'})
        .assign(predictedwin = lambda x: (x['newprediction']>0).astype(int))
        .assign(predictedloss = lambda x: (x['newprediction']<0).astype(int))
    ),
    (
        gamesim[['away_team','newprediction']].rename(columns={'away_team':'team'})
        .assign(predictedwin = lambda x: (x['newprediction']<0).astype(int))
        .assign(predictedloss = lambda x: (x['newprediction']>0).astype(int))
    )]).groupby('team')[['predictedwin','predictedloss']].sum().reset_index().assign(simnum=num)
    
    listofsim = pd.concat([listofsim,simulation])

In [660]:
#standard deviation of 13.74
#simulation.sort_values(by='predictedloss',ascending=True)

In [661]:
#simulation.query("team.isin(['LSU','Louisiana','Georgia','Michigan'])")

In [662]:
#listofsim.query("team=='LSU'").sort_values('predictedloss',ascending=True)
def q90(x):
    return x.quantile(.9)

def q10(x):
    return x.quantile(.1)

In [663]:
listofsim.groupby("team")[['predictedwin','predictedloss']].agg(['mean','median',q90,q10]).query("team=='Arkansas'")

predictedwin                  predictedloss                 
                 mean median  q90  q10          mean median  q90  q10
team                                                                 
Arkansas       5.3254    5.0  7.0  4.0        6.6746    7.0  8.0  5.0

In [664]:
pd.merge(
    listofsim.groupby('team')[['predictedwin','predictedloss']].mean().reset_index(),
    recap[['team','win','loss']],how='left',on='team').assign(missedwins = lambda x: x['win']-x['predictedwin']).sort_values('predictedwin',ascending=False)\
.sort_values('missedwins',ascending=False).head(20)
#.query("(missedwins>=1.5) or (missedwins<-1.5)")#.query("(missedwins<=1.5) and (missedwins>-1.5)")



,team,predictedwin,predictedloss,win,loss,missedwins
216,Washington,8.5714,3.4286,12,0,3.4286
78,Iowa,7.0638,4.9362,10,2,2.9362
141,Oklahoma State,6.1790,5.8210,9,3,2.8210
191,Toledo,8.2292,3.7708,11,1,2.7708
194,Tulane,8.3484,3.6516,11,1,2.6516
57,Florida State,9.6056,2.3944,12,0,2.3944
143,Ole Miss,7.6312,4.3688,10,2,2.3688
116,NC State,6.9138,5.0862,9,3,2.0862
91,Liberty,10.0454,1.9546,12,0,1.9546
184,Texas,9.1178,2.8822,11,1,1.8822


## Look at a team's distribution

In [668]:
display(pd.merge(
    listofsim.groupby('team')[['predictedwin','predictedloss']].mean().reset_index(),
    recap[['team','win','loss']],how='left',on='team').assign(missedwins = lambda x: x['win']-x['predictedwin']).sort_values('predictedwin',ascending=False)\
.query("team=='Washington'"))

display(listofsim.groupby(['team','predictedwin','predictedloss']).count().reset_index().assign(percent = lambda x: x['simnum']/5000)\
.query("team=='Washington'"))

        

,team,predictedwin,predictedloss,win,loss,missedwins
216,Washington,8.5714,3.4286,12,0,3.4286


,team,predictedwin,predictedloss,simnum,percent
1466,Washington,3,9,2,0.0004
1467,Washington,4,8,16,0.0032
1468,Washington,5,7,63,0.0126
1469,Washington,6,6,262,0.0524
1470,Washington,7,5,753,0.1506
1471,Washington,8,4,1261,0.2522
1472,Washington,9,3,1321,0.2642
1473,Washington,10,2,939,0.1878
1474,Washington,11,1,334,0.0668
1475,Washington,12,0,49,0.0098


In [667]:
#recap.assign(totalgames = lambda x: x['win']+x['loss']).query("totalgames < 5")['team']

#listofsim[listofsim['team'].isin(recap.assign(totalgames = lambda x: x['win']+x['loss']).query("totalgames < 5")['team'])]\
#.sort_values('predictedwin',ascending=False).head(20)

display(listofsim.groupby(['team','predictedwin','predictedloss']).count().reset_index().assign(percent = lambda x: x['simnum']/5000)\
.query("(predictedwin>=11)&(predictedloss==0)").query("percent>=.01").sort_values('percent',ascending=False))

,team,predictedwin,predictedloss,simnum,percent
985,Oregon,12,0,1247,0.2494
410,Georgia,12,0,1151,0.2302
721,Michigan,12,0,1027,0.2054
613,Liberty,12,0,520,0.1040
1003,Penn State,12,0,408,0.0816
29,Alabama,12,0,339,0.0678
386,Florida State,12,0,278,0.0556
601,LSU,12,0,243,0.0486
935,Ohio State,12,0,243,0.0486
54,Arizona,12,0,240,0.0480


In [583]:
listofsim.groupby('team')['predictedwin'].std().reset_index().sort_values('predictedwin',ascending=False).head(20)

,team,predictedwin
78,Iowa,1.594896
150,Purdue,1.583496
74,Illinois,1.574964
59,Fresno State,1.568458
118,Nebraska,1.566179
116,NC State,1.555376
55,Florida Atlantic,1.552508
40,Colorado State,1.546534
69,Houston,1.544922
213,Virginia Tech,1.541833


In [610]:
pd.merge(listofsim,recap[['team','win','loss']],how='left',on='team').assign(chaos=lambda x: abs(x['predictedwin']**2-x['win']**2))\
.groupby('simnum')['chaos'].sum().sort_values().head(10)

simnum
691    1986
950    2020
544    2034
29     2038
924    2044
383    2054
69     2070
558    2080
226    2082
901    2082
Name: chaos, dtype: int64

In [656]:
#Most "Chaotic Simulations"
#379,199,548,99,965,227,906 using doubled method
listofsim.query("simnum==454").sort_values(by='predictedwin',ascending=False).head(20)

,team,predictedwin,predictedloss,simnum
106,Michigan,12,0,454
146,Penn State,11,1,454
184,Texas,11,1,454
3,Alabama,11,1,454
140,Oklahoma,11,1,454
156,SMU,10,2,454
91,Liberty,10,2,454
88,LSU,10,2,454
191,Toledo,10,2,454
139,Ohio State,10,2,454
